<a href="https://colab.research.google.com/github/Mrshreesharma/Churn_pred_using_ANN/blob/main/Churn_prediction_using_ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
#!pip install tensorflow==2.12.0



In [28]:

import tensorflow as tf
print(tf.__version__)

2.18.0


In [29]:
# import basic libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [30]:
dataset = pd.read_csv('Churn_Modelling.csv')
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [31]:
## divide the dataset into independent and dependent feature.

X = dataset.iloc[:,3:13]
y = dataset.iloc[:,13]

In [32]:
## Feature Engineering
geography = pd.get_dummies(X['Geography'],drop_first=True)
gender = pd.get_dummies(X['Gender'],drop_first=True)

In [33]:
## Drop the encoded features
X.drop(['Geography','Gender'],axis=1,inplace=True)

In [34]:
X = pd.concat([X,geography,gender],axis = 1)

In [35]:
# Splitting the dataset into training set and test set

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=0)


In [36]:
# feature scaling

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [37]:
X_train.shape

(8000, 11)

In [38]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LeakyReLU,PReLU,ELU,ReLU
from tensorflow.keras.layers import Dropout

In [39]:
# Intializing the ANN

classifier = Sequential()

In [41]:
## Adding the Input Layer
classifier.add(Dense(units=10,activation='relu'))

In [75]:
# Adding the First Hidden Layer
classifier.add(Dense(units=7,activation='relu'))

# Adding the Second Hidden Layer
classifier.add(Dense(units=6,activation='relu'))

# Adding the Output Layer
classifier.add(Dense(units=1,activation='sigmoid'))

In [76]:
## updating the learning rate
opt = tf.keras.optimizers.Adam(learning_rate=0.01)

## Compiling the model.
classifier.compile(optimizer = opt,loss='binary_crossentropy',metrics=['accuracy'])



In [77]:
# Early Stopping

early_stop_var = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0.0001,
    patience=20,
    verbose=1,
    mode="auto",
    baseline=None,
    restore_best_weights=False,
)

In [78]:
model_history = classifier.fit(X_train,y_train,validation_split=0.33,batch_size=10,epochs=1000,callbacks=early_stop_var)

Epoch 1/1000
536/536 ━━━━━━━━━━━━━━━━━━━━ 10s 6ms/step - accuracy: 0.7870 - loss: 0.5387 - val_accuracy: 0.7955 - val_loss: 0.5073
Epoch 2/1000
536/536 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - accuracy: 0.7843 - loss: 0.5233 - val_accuracy: 0.7955 - val_loss: 0.5079
Epoch 3/1000
536/536 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - accuracy: 0.8118 - loss: 0.4839 - val_accuracy: 0.7955 - val_loss: 0.5068
Epoch 4/1000
536/536 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.7973 - loss: 0.5057 - val_accuracy: 0.7955 - val_loss: 0.5084
Epoch 5/1000
536/536 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.7931 - loss: 0.5107 - val_accuracy: 0.7955 - val_loss: 0.5077
Epoch 6/1000
536/536 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.7883 - loss: 0.5179 - val_accuracy: 0.7955 - val_loss: 0.5078
Epoch 7/1000
536/536 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.7957 - loss: 0.5088 - val_accuracy: 0.7955 - val_loss: 0.5068
Epoch 8/1000
536/536 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - accuracy: 0.7931 - loss: 0.5104 -

In [79]:
model_history.history.keys()

dict_keys(['accuracy', 'loss', 'val_accuracy', 'val_loss'])

In [80]:
## Predicting test results
y_pred = classifier.predict(X_test)
y_pred = (y_pred>=0.5)

63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step


In [81]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test,y_pred)
cm

array([[1595,    0],
       [ 405,    0]])

In [82]:
## Calculating the accuracy score
from sklearn.metrics import accuracy_score
score = accuracy_score(y_pred,y_test)
score

0.7975

In [83]:
## Getting the weights
classifier.get_weights()

[array([[ 1.3889748e-01,  2.9469170e-02,  1.2693654e+00, -4.5790121e-01,
          1.9151482e-01, -5.3332847e-01,  8.3724701e-01, -2.4681224e-01,
         -1.6430926e-01,  7.2434366e-01],
        [-2.6903899e+00,  2.2420496e-02, -3.7640510e+00,  2.7098212e+00,
         -1.9743766e-01,  7.3620933e-01, -3.0558629e+00,  1.1383027e+00,
          2.8176286e+00,  2.0731847e+00],
        [ 6.9470465e-02,  3.7402101e-02,  6.1770153e-01, -1.1622563e-01,
         -1.9488627e-02,  7.1634924e-01, -6.7458028e-01,  5.7934594e-01,
         -8.3739954e-01,  3.8365063e-01],
        [-1.0919074e+00, -1.6847862e-01, -6.7606407e-01,  7.3240739e-01,
         -1.2911466e+00,  3.4771630e-01, -6.1485684e-01, -2.7910354e+00,
         -1.2303630e+00, -7.4424553e-01],
        [ 1.9483557e-01,  3.5678053e+00, -8.3330065e-01, -1.6115417e+00,
          3.3465087e-01, -1.6422333e+00,  1.8847931e-02, -3.6351638e+00,
         -5.2667898e-01,  1.2786583e+00],
        [ 1.7943069e-01, -4.0475298e-02, -5.9452289e-01, -5.